# Yamanote Line Trajectory Map

##  Timestamped GeoJSON

In [1]:
import os

import numpy as np
import pandas as pd

import folium
from folium import plugins

print('folium version:', folium.__version__)

folium version: 0.10.1


In [2]:
df = pd.read_csv('山手線_shift.csv',
                 header = None,
                 names = ('station', 'dates', 'lat', 'long'),
                 encoding = 'SHIFT-JIS')
print(df)

# 日時データのみ抽出
df_date = df['dates']

# 緯度・経度データのみ抽出
df_lat = df['lat']
df_lon = df['long']

   station                dates        lat        long
0       大崎  2017-06-02T10:00:00  35.619700  139.728553
1      五反田  2017-06-02T11:00:00  35.626446  139.723444
2       目黒  2017-06-02T12:00:00  35.633998  139.715828
3      恵比寿  2017-06-02T13:00:00  35.646690  139.710106
4       渋谷  2017-06-02T14:00:00  35.658517  139.701334
5       原宿  2017-06-02T15:00:00  35.670168  139.702687
6      代々木  2017-06-02T16:00:00  35.683061  139.702042
7       新宿  2017-06-02T17:00:00  35.690921  139.700258
8     新大久保  2017-06-02T18:00:00  35.701306  139.700044
9     高田馬場  2017-06-02T19:00:00  35.712285  139.703782
10      目白  2017-06-02T20:00:00  35.721204  139.706587
11      池袋  2017-06-02T21:00:00  35.728926  139.710380
12      大塚  2017-06-02T22:00:00  35.731590  139.729329
13      巣鴨  2017-06-02T23:00:00  35.733492  139.739345
14      駒込  2017-06-03T00:00:00  35.736489  139.746875
15      田端  2017-06-03T01:00:00  35.738062  139.760860
16    西日暮里  2017-06-03T02:00:00  35.732135  139.766787
17     日暮里

In [3]:
# mapオブジェクトを定義
trajMap = folium.Map(
    # 新しい地図を作るときの中央にくる緯度と経度を入れる
    location = [35.6778658 , 139.73857073333332],
    # はじめに表示するときの縮尺を設定する
    zoom_start = 12
)

# linesを繰り返し代入する空のリストを定義
# for文の外で定義しなければならない
lines = []
# data frameの行数より1少ない値分繰り返す
for i in range(len(df) - 1):
    # 緯度・経度を'lines'に入れる(linesは辞書型リスト)
    line = [
        {
            # 座標点 *** 経度, 緯度の順番でリストに入れなければならない点に注意! ***
            'coordinates':[
                [df_lon.iloc[i], df_lat.iloc[i]], 
                [df_lon.iloc[i + 1], df_lat.iloc[i + 1]]
            ],
            # 日時
            'dates':[
                df_date.iloc[i],
                df_date.iloc[i + 1]
            ],
            # 線色
            'color': '#669DF6',
            # 線の太さ
            'weight': 5
        }
    ] # ----- line END -----
    lines.extend(line)

# featuresに入れる際に参照するlineオブジェクトを初期化
line = []
# featureを繰り返し代入するfeaturesリストを定義
# featuresの空のリストを定義
features = []
for line in lines:
    feature = [
        {
            'type': 'Feature',
            'geometry': {
                'type': 'LineString',
                'coordinates':line['coordinates'],
            },
            'properties': {
                'times': line['dates'],
                'style': {
                    'color': line['color'],
                    'weight': line['weight']
                }
            }
        }
    ] # ----- feature END -----
    features.extend(feature)

# folium plugin 
plugins.TimestampedGeoJson(
    {
        'type': 'FeatureCollection',
        'features': features,
    }, 
    period = 'PT1M', 
    add_last_point = True).add_to(trajMap)

# trajMap.save('trajMap.html')
trajMap